# Anhalyze Tutorial 


In this tutorial we'll take you to discover the capabilities of Anhalyze. 
In particular those ones from `AnhaDataset`, which is a python class that handles single `.nc` outputs from [NEMO](https://www.nemo-ocean.eu/) 
ocean model simulations with the [ANHA](https://canadian-nemo-ocean-modelling-forum-commuity-of-practice.readthedocs.io/en/latest/Institutions/UofA/Configurations/ANHA4/index.html) confinguration. 

We will be using tracer variables from NEMO ocean modules stored in gridT type files, 
as well as icebergs variables from the ICB module outputs.

We'll get these files by using `downloader`, an anhalyze module.

This tutorial assumes that you followed the installation instructions in the README.

#### Table of Contents:

* [Set Up](#set-up)
* [Example1: gridT](#example1)
    * [Static information](#static_info)
    * [Dynamic Functions](#methods)
* [Example2: icebergs](#example2)



### Set Up <a class="anchor" id="set-up"></a>


In [ ]:
%load_ext autoreload
%autoreload 2

# First we import anhalyze 
import anhalyze as ah

# ... and the downloader which 
import anhalyze.core.downloader as dw

# ... and other libraries we need
import os


In [ ]:
# This shows you how tu use this function
dw.download_example?

In [ ]:
# First, lets download the model output files we mentioned. 

# This downloads a gridT file as default.
dw.download_example()

# To download a different file you must specify the file_type. 
# The options are showed in the function help in the above output cell.
# This downloads the iceberg file.
dw.download_example(file_type='icebergs')

# The code should let you know when/if the download has been succesful. 

In [ ]:
# The example files have been downloaded to this location: 

ah.PACKAGE_DATA_DIR

# Which is within the Anhalyze installation directory. 

## Example 1: gridT <a class="anchor" id="example1"></a>

Lets use the gridT file to look at some temperature and sality data.


In [ ]:
# Listing the files in the directory above, we can check if there is
# a file with the string gridT in the name, then retreive the file name.
gridT_example_filename = [filename for filename in os.listdir(ah.PACKAGE_DATA_DIR) if 'gridT' in filename][0]
gridT_example_filename

In [ ]:
# Now we are ready to use `AnhaDataset` for the first time. 
# The class needs only the file name if located in the current directory, 
# otherwise one needs to provide the full/relative path to the file (as in this tutorial):
gridT_example = ah.AnhaDataset(ah.PACKAGE_DATA_DIR+'/'+gridT_example_filename)

# Technical note: 
# If you have not downloaded the mask file yet, it will be automatically downloaded at this point.

In [ ]:
# Here you can find more information about how to use `AnhaDataset`.

ah.AnhaDataset?

# Technical note: 
# If you are familiar with the library `xarray` to open `.nc` files.
# You can see here `Anhadataset` is kind of a wrapper to `xarray.Dataset` with specific implemetation for ANHA files. 


### Static information <a class="anchor" id="static_info"></a>

We can then we can look at the information in the file:


In [ ]:
# The dimentions can be accessed like this:
gridT_example.dims

In [ ]:
# The dictionary of Anhalyze attributes are shown like this: 
gridT_example.attrs

# Technical note: 
# This contains the original attributes found in `xarray.Dataset.attrs`, and includes more information specific to Anhalyze.

In [ ]:
# The coordinates can be viewed like this:
gridT_example.coords


In [ ]:
# The data variables and their general information can be displayed like this:
gridT_example.data_vars


In [ ]:
# You can also use this nice html representation which gives you more flexibility by allowing you 
# to manipulate the way you see all the information above with the exeption of attrs, 
# which only shows the original information in the file. 

gridT_example

### Dynamic Functions <a class="anchor" id="methods"></a>

The example file was open with `AnhaDataset` and is then an *instance* of that python class. As such, besided the static information (class *attributes*), it also contains functions (class *methods*).


#### Data Selection

In [ ]:
# We can select a region based on given ranges of latitude(degrees North), longitude(degrees East) and/or depth (positive). 
# We give values in the specified units.

gridT_example.sel?

In [ ]:
# We can also select a region based on given ranges of latitude, longitude and depth. 
# But using grid indexing, indicating i, j, and k positions.

gridT_example.isel?

In [ ]:
# We'll then select a region by given latitude, longitude and depth ranges. 
# Let's create a new AnhaDataset with the selection:

gridT_example_region = gridT_example.sel(lat_range=[50,65],lon_range=[-93,-75],depth_range=[0,300])

# Note you can also call it like this:  
# gridT_example_region = gridT_example.sel([50,65],[-93,-75],[0,300])
# as long as you give latitude, longitude and depth range in that order.


In [ ]:
# You can inspect the selected region attributes like this:

gridT_example_region

#### File input/output (I/O)

In [ ]:
# This new AnhaDataset can be saved to disk as netcdf file 
# with the following command

gridT_example_region.to_netcdf?

# We caution here that contratry to xarray.Dataset.to_netcdf we avoid overwriting files.
# The function adds a suffix to the new netcdf file, which can be chosen by the user.

In [ ]:

# This new dataset can be saved to disk:
gridT_example_region.to_netcdf()

# .. and open again by adding the defuault suffix
region_filename = ah.PACKAGE_DATA_DIR+'/'+gridT_example_filename.replace('.nc','_CutRegion.nc')
gridT_example_region_from_file = ah.AnhaDataset(region_filename)


In [ ]:
# Again, you can inspect the attributes like this:

gridT_example_region_from_file

#### Mapping

In [ ]:
# The last but not least Anhalyze tool is to plot spatial data
# into a projected map, creating figures in a vary simple way.

gridT_example_region.show_var_data_map?


In [ ]:
# Here we show the sea surface temperature, and you can see the region we selected is Hudson Bay!

gridT_example_region_from_file.show_var_data_map(var='votemper')

# You might be wondering: I gave a depth range, what is the plotting showing?
# For now, the function only plots the first selected layer. 

In [ ]:
# we can change the color range and projection like this:

gridT_example_region_from_file.show_var_data_map(var='votemper', color_range='relative', projection_name='Mercator')

In [ ]:
# The color palette uses the cmocean library 
# which is a package to help standardize colormaps for commonly-plotted oceanographic properties.

# Here we show salinity with the respective colormap. 

gridT_example_region_from_file.show_var_data_map(var='vosaline', projection_name='Robinson')

In [ ]:
# finally we can save the figure to file:

savefig_filename = gridT_example_region_from_file.attrs['filename'].replace('.nc','.png')
gridT_example_region_from_file.show_var_data_map(var='vosaline',savefig=savefig_filename)


## Example 2: icebergs <a class="anchor" id="example2"></a>


We can also open iceberg and icemod files! 
Lets use the icebergs file we downloaded at the begining of the tutorial. 


In [ ]:
# Now lets check the icebergs example

icebergs_example_filename = [filename for filename in os.listdir(ah.PACKAGE_DATA_DIR) if 'icebergs' in filename][0]
icebergs_example_filename


In [ ]:
# we open the file the same way
# we have almost all the functionallity avaialble except for depth. 

icebergs_example = ah.AnhaDataset(ah.PACKAGE_DATA_DIR+'/'+icebergs_example_filename)
icebergs_example

In [ ]:
# We can look at the melting rate for instance:

icebergs_example.data_vars['berg_floating_melt']

In [ ]:

# we first select a region
icebergs_example_region = icebergs_example.sel(lat_range=[55,80],lon_range=[-65,-55])

# and then we can map it
icebergs_example_region.show_var_data_map('berg_floating_melt', projection_name='Mercator')
